In [ ]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# print gpu info
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Define the Q-Network model
def create_q_network(input_shape, action_space):
    model = Sequential(
        [
            tf.keras.Input(shape=input_shape),
            Dense(32, activation='relu'),
            Dense(32, activation='relu'),
            Dense(action_space, activation='linear')
        ]
    )
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

In [ ]:
def euclidean_distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

In [ ]:
class ENV():
    def __init__(self):
        self.steps = 0
        self.env_col = 11
        self.env_row = 11
        self.state = (5, 5)
        self.action_space = 4
        self.reward = 0
        self.done = False
        self.goal = (1, 1)

    def reset(self):
        # self.goal = np.random.randint(0, min(self.env_col, self.env_row), size=2)
        self.state = np.random.randint(0, min(self.env_col, self.env_row), size=2)
        self.reward = 0
        self.done = False
        self.steps = 0
        return self.state
    
    def step(self, action):
        prev_state = self.state
        self.steps += 1
        if action == 0: # up
            self.state = (self.state[0], self.state[1] + 1)
        elif action == 1: # down
            self.state = (self.state[0], self.state[1] - 1)
        elif action == 2: # left
            self.state = (self.state[0] - 1, self.state[1])
        elif action == 3: # right
            self.state = (self.state[0] + 1, self.state[1])
        else:
            raise ValueError("Invalid action")
        # print(type(self.state), type(self.goal)) # debug
        if tuple(self.state) == tuple(self.goal):
            self.reward = 100
            self.done = True
            # self.goal = np.random.randint(0, min(self.env_col, self.env_row), size=2)
        elif self.state[0] < 0 or self.state[0] >= self.env_col or self.state[1] < 0 or self.state[1] >= self.env_row:
            self.reward = -100
            self.done = True
        elif self.steps >= 20:
            self.reward = -100
            self.done = True
        else:
            if euclidean_distance(self.state, self.goal) <= euclidean_distance(prev_state, self.goal):
                self.reward = 1
            else:
                self.reward = -1
        return self.state, self.reward, self.done


In [ ]:
#Define the DQN agent
class DQNAgent:
    def __init__(self, state_shape, action_space, goal):
        self.goal = goal
        self.state_shape = state_shape
        self.action_space = action_space
        self.memory = []
        self.gamma = 0.95  # Discount factor
        self.epsilon = 0.9  # Exploration rate
        self.epsilon_decay = 0.995  # Decay rate for exploration rate
        self.epsilon_min = 0.01  # Minimum exploration rate
        # self.model = create_q_network(state_shape, action_space) # state_shape = (2,)
        try:
            self.model = tf.keras.models.load_model('model_dqn_pt_to_pt_diff_start_diff_goal.h5') # loading the last saved model
        except:
            self.model = create_q_network((4,), action_space)

    def remember(self, state, goal, action, reward, next_state, done):
        self.memory.append((state, goal, action, reward, next_state, done))
        if len(self.memory) > 200:
            self.memory = self.memory[-200:]

    def act(self, state, test=False):
        if not test:
            if np.random.rand() < self.epsilon:
                return np.random.choice(self.action_space)
        # print("state in action: ",state)
        state_goal = np.concatenate((state, self.goal))
        state_goal = np.reshape(state_goal, (4,))
        state_goal = tf.convert_to_tensor(state_goal)
        state_goal = tf.expand_dims(state_goal, 0)
        # print("Predicting")
        q_values = self.model.predict(state_goal, verbose=0)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        # batch = np.random.choice(self.memory, batch_size, replace=False)
        batch_ch = np.random.choice(len(self.memory), batch_size, replace=False)
        batch = [self.memory[i] for i in batch_ch]
        for state, goal, action, reward, next_state, done in batch:
            state = np.reshape(state, self.state_shape)
            next_state = np.reshape(next_state, self.state_shape)
            target = reward
            if not done:
                # print(next_state.shape)
                next_state_goal = np.concatenate((next_state, self.goal))
                next_state_goal = tf.convert_to_tensor(next_state_goal)
                next_state_goal = tf.expand_dims(next_state_goal, 0)
                target = reward + self.gamma * np.amax(self.model.predict(next_state_goal, verbose=0)[0])
            # print(state.shape)
            state_goal = np.concatenate((state, self.goal))
            state_goal = tf.convert_to_tensor(state_goal)
            state_goal = tf.expand_dims(state_goal, 0)
            # print("Predicting")
            target_f = self.model.predict(state_goal, verbose=0)
            target_f[0][action] = target
            # print(state_goal[0], state_goal[1])
            # print("Fitting model")
            self.model.fit(state_goal, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
# Initialize the environment and agent
state_shape = (2,)  # Example state shape, adjust according to your actual state representation # Only x and y coordinates
action_space = 4  # Example action space size, adjust according to your actual actions # 4 actions: up, down, left, right
env = ENV()
agent = DQNAgent(state_shape, action_space, env.goal)

In [ ]:
total_reward = []

In [ ]:
#Training loop
num_episodes = 50  # Set the number of training episodes
batch_size = 32  # Set the batch size for replay
start_ep = 0 # Set the starting episode number
for episode in range(start_ep, start_ep + num_episodes):
    print("Episode: ", episode)
    state = env.reset()
    state = np.reshape(state, state_shape)
    done = False
    episode_reward = 0
    while not done:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, state_shape)
        agent.remember(state, env.goal, action, reward, next_state, done)
        state = next_state
        episode_reward += reward
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    # if agent.epsilon > agent.epsilon_min:
    #     agent.epsilon *= agent.epsilon_decay
    if tuple(state) == tuple(env.goal): # If the agent reaches the goal position, we update the goal position
        env.goal = np.random.randint(0, min(env.env_col, env.env_row), size=2)
        agent.goal = env.goal
    total_reward.append(episode_reward)
    print("Episode reward: ", episode_reward)
    if agent.epsilon > agent.epsilon_min: # Decay the exploration rate
        agent.epsilon *= agent.epsilon_decay
    if episode%10 == 0: # Save the model every 10 episodes
        print("Saving model")
        agent.model.save(f"model_dqn_pt_to_pt_diff_start_diff_goal_{episode}.h5")
    agent.model.save(f"model_dqn_pt_to_pt_diff_start_diff_goal.h5") # Save the model after each episode



In [ ]:
# Use the trained agent to control the agent in the environment
state = env.reset()
print("state: ", state)
print("goal: ", env.goal)
state = np.reshape(state, state_shape)
done = False
while not done:
    action = agent.act(state, test=True)
    next_state, _, done = env.step(action)
    next_state = np.reshape(next_state, state_shape)
    # Take action with the agent in the environment
    state = next_state
    print(state)

In [ ]:
agent.model.save("model_dqn_pt_to_pt_diff_start_diff_goal.h5")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(total_reward)
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()

In [ ]:
from guppy import hpy
h = hpy()
print(h.heap())